# Fast tokenizers in the QA pipeline (PyTorch)

Installez les bibliothèques 🤗 *Transformers* et 🤗 *Datasets* pour exécuter ce *notebook*.

In [ ]:
!pip install datasets transformers[sentencepiece]

In [ ]:
from transformers import pipeline

question_answerer = pipeline("question-answering", model="etalab-ia/camembert-base-squadFR-fquad-piaf")
context = """
🤗 Transformers s'appuie sur les trois bibliothèques d'apprentissage profond les plus populaires (Jax, PyTorch et TensorFlow) avec une intégration transparente entre elles.
C'est simple d'entraîner vos modèles avec l'une avant de les charger pour l'inférence avec l'autre.
"""
question = "Quelles bibliothèques d'apprentissage profond derrière 🤗 Transformers ?"
question_answerer(question=question, context=context)

In [ ]:
long_context = """
🤗 Transformers : l'état de l'art du NLP

🤗 Transformers fournit des milliers de modèles pré-entraînés pour effectuer des tâches sur des textes telles que la classification, 
l'extraction d'informations, la réponse à des questions, le résumé de textes, la traduction, la génération de texte et plus encore dans plus de 100 langues.
Son objectif est de rendre le traitement automatique des langues de pointe plus facile à utiliser pour tout le monde.

🤗 Transformers fournit des API permettant de télécharger et d'utiliser rapidement ces modèles pré-entraînés sur un texte donné, de les affiner sur vos propres ensembles de données et de les partager avec la communauté sur notre site Web.
puis de les partager avec la communauté sur notre hub de modèles. En même temps, chaque module python définissant une architecture est entièrement autonome et peut être modifié pour permettre des expériences de recherche rapides.
peut être modifié pour permettre des expériences de recherche rapides.

Pourquoi devrais-je utiliser des transformateurs ?

1. Des modèles de pointe faciles à utiliser :
  - Haute performance sur les tâches NLU et NLG.
  - Faible barrière à l'entrée pour les éducateurs et les praticiens.
  - Peu d'abstractions pour l'utilisateur avec seulement trois classes à apprendre.
  - Une API unifiée pour utiliser tous nos modèles pré-entraînés.
  - Des coûts de calcul plus faibles, une empreinte carbone réduite :

2. Les chercheurs peuvent partager les modèles formés au lieu de toujours les reformer.
  - Les praticiens peuvent réduire le temps de calcul et les coûts de production.
  - Des dizaines d'architectures avec plus de 10 000 modèles pré-formés, certains dans plus de 100 langues.

3. Choisissez le cadre approprié pour chaque étape de la vie d'un modèle :
  - Entraînez des modèles de pointe en 3 lignes de code.
  - Déplacez un seul modèle entre les frameworks TF2.0/PyTorch à volonté.
  - Choisissez de manière transparente le bon framework pour l'entraînement, l'évaluation et la production.

4. Adaptez facilement un modèle ou un exemple à vos besoins :
  - Nous fournissons des exemples pour chaque architecture afin de reproduire les résultats publiés par ses auteurs originaux.
  - Les éléments internes des modèles sont exposés de manière aussi cohérente que possible.
  - Les fichiers de modèles peuvent être utilisés indépendamment de la bibliothèque pour des expériences rapides.

🤗 Transformers s'appuie sur les trois bibliothèques d'apprentissage profond les plus populaires (Jax, PyTorch et TensorFlow) avec une intégration parfaite
entre elles. Il est simple d'entraîner vos modèles avec l'une avant de les charger pour l'inférence avec l'autre.
"""
question_answerer(question=question, context=long_context)

In [ ]:
from transformers import AutoTokenizer, AutoModelForQuestionAnswering

model_checkpoint = "etalab-ia/camembert-base-squadFR-fquad-piaf"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = AutoModelForQuestionAnswering.from_pretrained(model_checkpoint)

inputs = tokenizer(question, context, return_tensors="pt")
outputs = model(**inputs)

In [ ]:
start_logits = outputs.start_logits
end_logits = outputs.end_logits
print(start_logits.shape, end_logits.shape)

In [ ]:
import torch

sequence_ids = inputs.sequence_ids()
# Masque tout sauf les tokens du contexte
mask = [i != 1 for i in sequence_ids]
# Démasque le token [CLS]
mask[0] = False
mask = torch.tensor(mask)[None]

start_logits[mask] = -10000
end_logits[mask] = -10000

In [ ]:
start_probabilities = torch.nn.functional.softmax(start_logits, dim=-1)[0]
end_probabilities = torch.nn.functional.softmax(end_logits, dim=-1)[0]

In [ ]:
scores = start_probabilities[:, None] * end_probabilities[None, :]

In [ ]:
scores = torch.triu(scores)

In [ ]:
max_index = scores.argmax().item()
start_index = max_index // scores.shape[1]
end_index = max_index % scores.shape[1]
print(scores[start_index, end_index])

In [ ]:
inputs_with_offsets = tokenizer(question, context, return_offsets_mapping=True)
offsets = inputs_with_offsets["offset_mapping"]

start_char, _ = offsets[start_index]
_, end_char = offsets[end_index]
answer = context[start_char:end_char]

In [ ]:
result = {
    "answer": answer,
    "start": start_char,
    "end": end_char,
    "score": scores[start_index, end_index],
}
print(result)

In [ ]:
inputs = tokenizer(question, long_context)
print(len(inputs["input_ids"]))

In [ ]:
inputs = tokenizer(question, long_context, max_length=384, truncation="only_second")
print(tokenizer.decode(inputs["input_ids"]))

In [ ]:
sentence = "Cette phrase n'est pas trop longue mais nous allons la diviser quand même."
inputs = tokenizer(
    sentence, truncation=True, return_overflowing_tokens=True, max_length=6, stride=2
)

for ids in inputs["input_ids"]:
    print(tokenizer.decode(ids))

In [ ]:
print(inputs.keys())

In [ ]:
print(inputs["overflow_to_sample_mapping"])

In [ ]:
sentences = [
    "Cette phrase n'est pas trop longue mais on va quand même la diviser.",
    "Cette phrase est plus courte mais sera quand même divisée.",
]
inputs = tokenizer(
    sentences, truncation=True, return_overflowing_tokens=True, max_length=6, stride=2
)

print(inputs["overflow_to_sample_mapping"])

In [ ]:
inputs = tokenizer(
    question,
    long_context,
    stride=128,
    max_length=384,
    padding="longest",
    truncation="only_second",
    return_overflowing_tokens=True,
    return_offsets_mapping=True,
)

In [ ]:
_ = inputs.pop("overflow_to_sample_mapping")
offsets = inputs.pop("offset_mapping")

inputs = inputs.convert_to_tensors("pt")
print(inputs["input_ids"].shape)

In [ ]:
outputs = model(**inputs)

start_logits = outputs.start_logits
end_logits = outputs.end_logits
print(start_logits.shape, end_logits.shape)

In [ ]:
sequence_ids = inputs.sequence_ids()
# Masque tout sauf les tokens du contexte.
mask = [i != 1 for i in sequence_ids]
# Démasquer le token [CLS]
mask[0] = False
# Masquer tous les tokens [PAD]
mask = torch.logical_or(torch.tensor(mask)[None], (inputs["attention_mask"] == 0))

start_logits[mask] = -10000
end_logits[mask] = -10000

In [ ]:
start_probabilities = torch.nn.functional.softmax(start_logits, dim=-1)
end_probabilities = torch.nn.functional.softmax(end_logits, dim=-1)

In [ ]:
candidates = []
for start_probs, end_probs in zip(start_probabilities, end_probabilities):
    scores = start_probs[:, None] * end_probs[None, :]
    idx = torch.triu(scores).argmax().item()

    start_idx = idx // scores.shape[0]
    end_idx = idx % scores.shape[0]
    score = scores[start_idx, end_idx].item()
    candidates.append((start_idx, end_idx, score))

print(candidates)

In [ ]:
for candidate, offset in zip(candidates, offsets):
    start_token, end_token, score = candidate
    start_char, _ = offset[start_token]
    _, end_char = offset[end_token]
    answer = long_context[start_char:end_char]
    result = {"answer": answer, "start": start_char, "end": end_char, "score": score}
    print(result)